# Einführung in das Machine Learning (Woche 3): Von Daten zu Vorhersagen

In dieser Vorlesung werden Sie lernen, wie Sie von einem Beispieldatensatz im csv-Format zu einem ersten Machine-Learning Modell gelangen.

## Jupyter und Jupyter Notebooks

Jupyter ist ein Framework zum interaktiven Programmieren. Letzte Woche haben Sie bereits im Tutorium Python und Pip aufgesetzt. Deshalb ist es nun ganz einfach, dass Sie auch Jupyter aufsetzen, installieren Sie es einfach via:

``` pip install jupyter ```

In Pycharm können Sie ein Notebook lokal starten (dies habe ich getan; [offizielle Hilfeseite für Pycharm](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html)), ansonsten führen Sie einfach:

``` jupyter lab ```

um es im Browser zu starten. Bei Problemen konsultieren Sie bitte zuerst die Dokumentation von Jupyter ([Dokumentation](https://docs.jupyter.org/en/latest/)) und fragen, wenn die Probleme weiter bestehen, im Tutorium.

Der Vorteil von Jupyter Notebooks gegenüber reinem Scripting in nativem Python ist, dass Sie Zwischenergebnisse sich anzeigen lassen können und interaktiv schnell Feedback auf kleine Code-Snippets bekommen können. Wir verwenden es hier in der Vorlesung für Lernzwecke. Ich möchte darauf hinweisen, dass Sie Jupyter auch in der Industrie zum schnellen Testen von Ideen nutzen können oder um Kunden etwas zu präsentieren (Stichwort: Fast Prototyping), allerdings ist es zum Produktivbetrieb regelmäßig ungeeignet, da die interaktive Engine (der Jupyter-Server) im Hintergrund die Ausführung Ihres Programmes verlangsamt.

In [ ]:
print("Hello World!")


In [ ]:
%pip install pandas
import pandas
from pandas import DataFrame # Das DataFrame (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) ist die zentrale In-Memory Struktur in Pandas. Sie können es sich wie eine kleine Excel-Tabelle oder eine relationale Datenbank in Ihrem Hauptspeicher vorstellen.

In [ ]:
import os

df = pandas.read_csv(os.path.join("titanic_data", "train.csv"))

# Ignorieren Sie für den Moment die test.csv und test_transformed.csv. Wir gehen später darauf ein, warum es sinnvoll ist, zwischen Trainings- und Testdatensatz zu unterscheiden.

df.head(10)

In [ ]:
target = df["Survived"]
df = df.drop(columns=["Survived"])
df.head(10)

In [ ]:
target

In [ ]:
df = df.drop(columns=["PassengerId", "Name", "Ticket"])

sex_one_hot = pandas.get_dummies(df["Sex"], dtype="int")
embarked_one_hot = pandas.get_dummies(df["Embarked"], dtype="int")

sex_one_hot

In [ ]:
embarked_one_hot

In [ ]:
df = df.drop(columns = ["Sex", "Embarked"]) # Es gibt kein Grund die gleiche Information zweimal im Datensatz codiert zu halten
df = df.join(sex_one_hot)
df = df.join(embarked_one_hot)

df.head(10)

In [ ]:
df = df.drop(columns = ["male", "S"])

In [ ]:
fares = df["Fare"]
passenger_classes = df["Pclass"]

passenger_class_1_indices = df.index[df['Pclass'] == 1]
passenger_class_2_indices = df.index[df['Pclass'] == 2]
passenger_class_3_indices = df.index[df['Pclass'] == 3]

fares_class_1 = fares[passenger_class_1_indices]
fares_class_2 = fares[passenger_class_2_indices]
fares_class_3 = fares[passenger_class_3_indices]

print(f"Tickets for class 1 were sold at max for: {fares_class_1.max()} and at min for {fares_class_1.min()}")
print(f"Tickets for class 2 were sold at max for: {fares_class_2.max()} and at min for {fares_class_2.min()}")
print(f"Tickets for class 3 were sold at max for: {fares_class_3.max()} and at min for {fares_class_3.min()}")

In [ ]:
df.head(10)

In [ ]:
cabin = df["Cabin"]
age = df["Age"]

print(f"NaN share in Cabin: {cabin.isna().sum()/len(cabin)*100:.2f} %")
print(f"NaN share in Age: {age.isna().sum()/len(cabin)*100:.2f} %")

In [ ]:
%pip install numpy
import numpy

In [ ]:
df = df.drop(columns=["Cabin"])
age = age.to_numpy() # Übergang von Pandas zu Numpy über explizites Interface

In [ ]:
age

In [ ]:
print(f"The minimum age is: {numpy.nanmin(age)}") # Nanmin ignoriert alle NaN Werte. Einfach nur "min" zu rufen, würde hingegen in der Rückgabe NaN resultieren.
print(f"For reference: {numpy.min(age)}") # Das Gleiche gilt für die weiteren Funktionen.
print(f"The maximum age is: {numpy.nanmax(age)}")
print(f"The mean value is: {numpy.nanmean(age)}") # Der Mean muss kein Element der Daten selber sein!
print(f"The median age is: {numpy.nanmedian(age)}")
vals,counts = numpy.unique(age[~numpy.isnan(age)], return_counts=True) # ~numpy.isnan(age) exkludiert für einen Moment alle NaN-Werte. ~ ist die Negation.
print(f"The mode age is: {vals[numpy.argmax(counts)]}") # Der Mode lässt sich leider noch immer nicht komfortabel in einem Schritt berechnen
print(f"For reference: All age values \n {vals} \n with their respective counts \n {counts}")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(age, bins=len(counts))
print(f"The standard deviation is: {numpy.nanstd(age)}")

In [ ]:
age = numpy.nan_to_num(age, copy=True, nan=numpy.nanmedian(age))
df["Age"] = age # Übergang von Numpy zu Pandas über implizites Interface. Gefahr!
df.head(10)

In [ ]:
%pip install scikit-learn
import sklearn

In [ ]:
from sklearn import tree

data = df.to_numpy()
clf = tree.DecisionTreeClassifier(max_depth=5) # Maximal 5 Wenn-Dann-Fragen
clf.fit(data, target)

In [ ]:
from matplotlib import pyplot

print("Visualization of the Decision Tree:")
pyplot.figure(figsize=(12, 12))
tree.plot_tree(clf, feature_names=df.columns, max_depth=2)
pyplot.show()

In [ ]:
r = tree.export_text(clf, feature_names=df.columns)
print("Verbal Description of Decision Tree")
print(r)

In [ ]:
df_test = pandas.read_csv(os.path.join("titanic_data", "test.csv"))
df_test_transformed = pandas.read_csv(os.path.join("titanic_data", "test_transformed.csv"))

df_test_transformed.head(10)

In [ ]:
ground_truth = df_test["Survived"].to_numpy()
data_test = df_test_transformed.to_numpy()
predicted_truth = clf.predict(data_test)

In [ ]:
ground_truth

In [ ]:
predicted_truth

In [ ]:
print(sklearn.metrics.classification_report(ground_truth, predicted_truth))

## Abschluss

Herzlichen Glückwunsch! Sie haben erfolgreich Ihre erste Machine-Learning-Pipeline durchgestanden und ein 77% der ungesehenen Beispiele richtig erraten.

Damit schließen wir die heutige Vorlesung. Für weitere Fragen kontaktieren Sie mich gerne unter tim.barz-cech@th-luebeck.de. Ansonsten freue ich mich wie immer über ihr Feedback und wir sehen uns beim nächsten Termin! Wie bereits geübt gilt: Sie haben diese Vorlesung aller Wahrscheinlichkeit nach verstanden, wenn Sie alle Fragen selbstständig ohne Hilfsmittel beantworten können. Bitte wiederholen Sie die Vorlesung frühzeitig, damit Sie nicht zur Prüfungszeit in zu großen Stress geraten.

Solange haben Sie eine schöne Zeit!